In [16]:
import cv2
import numpy as np
import tensorflow as tf

In [17]:
# Load the trained CNN model
model = tf.keras.models.load_model('path/to/your/model.h5')

In [18]:
# Define a function to preprocess the input image
def preprocess(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Threshold the image to make it binary
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # Find the contours in the image
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Find the bounding box of the largest contour
    x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
    # Crop the image to the bounding box
    image_cropped = binary[y:y + h, x:x + w]
    # Resize the image to 28x28
    image_resized = cv2.resize(image_cropped, (28, 28), interpolation=cv2.INTER_AREA)
    # Reshape the image to 4D tensor (batch_size, height, width, channels)
    image_tensor = np.reshape(image_resized, (1, 28, 28, 1))
    # Convert the image to float32 and scale it to [0, 1]
    image_tensor = image_tensor.astype('float32') / 255.0
    return image_tensor


In [19]:
# Define a function to recognize the digit in the input image
def recognize_digit(image):
    # Preprocess the input image
    image_tensor = preprocess(image)
    # Use the trained CNN model to predict the digit in the image
    predicted_probabilities = model.predict(image_tensor)
    # Get the predicted class with highest probability
    predicted_digit = np.argmax(predicted_probabilities, axis=1)[0]
    return predicted_digit


In [22]:
# Open the webcam
cap = cv2.VideoCapture(0)

# Loop until the 'f' key is pressed
while True:
    # Read the current frame from the webcam
    ret, frame = cap.read()
    # Display the current frame
    cv2.imshow('Webcam', frame)
    # Check if the 'f' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('f'):
        break

# Release the webcam
cap.release()

# Convert the image to grayscale
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Recognize the digit in the input image
predicted_digit = recognize_digit(frame)

# Display the predicted digit
cv2.putText(frame, str(predicted_digit), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
cv2.imshow('Recognized Digit', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 62ms/step
